# ESPnet2 real streaming Transformer demonstration
This local notebook provides a demonstration of streaming ASR based on Transformer using ESPnet2.

You can recognize a recorded audio file or a speech online.

Author: Keqi Deng (UCAS)

## Download pre-trained model and audio file for demo
Please download and unzip the following files from the Google Drive to the same directory as this local notebook.
- https://drive.google.com/file/d/1i4N--kh-QvoJ974F22cWAIV5Tzazv7cx/view?usp=sharing    (prepare status)
- https://drive.google.com/file/d/1xHfVJ1YRoDDvvt2ADg58-alQvJdp4eFi/view?usp=sharing    (download pretrained Mandarin model)
- https://drive.google.com/file/d/1bLXeF2zUDi2G2jBwslqX9UMHh7H1FPLn/view?usp=sharing    (download corresponding config)
- https://drive.google.com/file/d/1l8w93r8Bs5FtC3A-1ydEqFQdP4k6FiUL/view?usp=sharing    (download audio file for demo)

## Import packages

In [1]:
import sys
import espnet
from espnet2.bin.asr_inference_streaming import Speech2TextStreaming
import argparse
import numpy as np
import wave

## Prepare for inference

In [2]:
asr_train_config='./config.yaml'
asr_model_file='./valid.acc.ave.pth'
speech2text = Speech2TextStreaming(
    asr_train_config=asr_train_config,
    asr_model_file=asr_model_file,
    lm_train_config=None,
    lm_file=None,
    token_type=None,
    bpemodel=None,
    maxlenratio=0.0,
    minlenratio=0.0,
    beam_size=20,
    ctc_weight=0.3,
    lm_weight=0.0,
    penalty=0.0,
    nbest=1,
    device = "cpu",
    disable_repetition_detection=True,
    decoder_text_length_limit=0,
    encoded_feat_length_limit=0
)

In [3]:
prev_lines = 0
def progress_output(text):
    global prev_lines
    lines=['']
    for i in text:
        if len(lines[-1]) > 50:
            lines.append('')
        lines[-1] += i
    for i,line in enumerate(lines):
        if i == prev_lines:
            sys.stderr.write('\n\r')
        else:
            sys.stderr.write('\r\033[B\033[K')
        sys.stderr.write(line)

    prev_lines = len(lines)
    sys.stderr.flush()

In [4]:
def recognize(wavfile):
    with wave.open(wavfile, 'rb') as wavfile:
        ch=wavfile.getnchannels()
        bits=wavfile.getsampwidth()
        rate=wavfile.getframerate()
        nframes=wavfile.getnframes()
        buf = wavfile.readframes(-1)
        data=np.frombuffer(buf, dtype='int16')
    speech = data.astype(np.float16)/32767.0
    sim_chunk_length = 512
    if sim_chunk_length > 0:
        for i in range(len(speech)//sim_chunk_length):
            results = speech2text(speech=speech[i*sim_chunk_length:(i+1)*sim_chunk_length], is_final=False)
            if results is not None and len(results) > 0:
                nbests = [text for text, token, token_int, hyp in results]
                text = nbests[0] if nbests is not None and len(nbests) > 0 else ""
                progress_output(nbests[0])
            else:
                progress_output("")
            
        results = speech2text(speech[(i+1)*sim_chunk_length:len(speech)], is_final=True)
    else:
        results = speech2text(speech, is_final=True)
    nbests = [text for text, token, token_int, hyp in results]
    progress_output(nbests[0])

## Recognize the audio file

In [5]:
wavfile='./BAC009S0724W0121.wav'
recognize(wavfile)

c:\programdata\anaconda3\envs\tf\lib\site-packages\torch\functional.py:573: UserWarning: stft will soon require the return_complex parameter be given for real inputs, and will further require that return_complex=True in a future PyTorch release. (Triggered internally at  ..\aten\src\ATen\native\SpectralOps.cpp:659.)
  normalized, onesided, return_complex)
c:\programdata\anaconda3\envs\tf\lib\site-packages\espnet2\layers\stft.py:109: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  olens = (ilens - self.win_length) // self.hop_length + 1

c:\programdata\anaconda3\envs\tf\lib\site-packages\espnet\nets\batch_beam_search.py:107: UserWarning: __floordiv__ is deprecate

## Recognize the speech from speaker

### Install pyaudio

In [6]:
import pyaudio

### Streamingly recognize with pyaudio

In [8]:
CHUNK=2048
FORMAT=pyaudio.paInt16
CHANNELS=1
RATE=16000
RECORD_SECONDS=5
p=pyaudio.PyAudio()
stream = p.open(format=FORMAT,channels=CHANNELS,rate=RATE,input=True,frames_per_buffer=CHUNK)
for i in range(0,int(RATE/CHUNK*RECORD_SECONDS)+1):
    data=stream.read(CHUNK)
    data=np.frombuffer(data, dtype='int16')
    data=data.astype(np.float16)/32767.0
    if i==int(RATE/CHUNK*RECORD_SECONDS):
        results = speech2text(speech=data, is_final=True)
        break
    results = speech2text(speech=data, is_final=False)
    if results is not None and len(results) > 0:
        nbests = [text for text, token, token_int, hyp in results]
        text = nbests[0] if nbests is not None and len(nbests) > 0 else ""
        progress_output(nbests[0])
    else:
        progress_output("")
nbests = [text for text, token, token_int, hyp in results]
progress_output(nbests[0])

想了解更多马上注册